# Create 3D maps in ArcGIS Pro with Lidar data
ArcGIS Pro Python script to process LAS and NAIP layers in preparation for 3D analysis and visualization. This script is for Kentucky, y'all!

In [1]:
# Import ArcGIS package
import arcpy

# Subprocess allows us to issue commands on the command line
import subprocess

# Module to download files with an URL
from urllib.request import urlretrieve

# Zip utlity to extract files
from zipfile import ZipFile

# Utility to compress folder
import shutil

# pandas to view table and possibly analyze data in the future?
import pandas as pd

## Instructions

* Pick a location anywhere in Kentucky and drop a point in ArcGIS.
* Download and extract [download_grids.gdb.zip](https://uky-gis.github.io/support/python-arcgis/pointcloud_extract/download_grids.gdb.zip)
* Download the laszip64.exe utility

## Example

![Cumberland Falls](https://uky-gis.github.io/support/python-arcgis/pointcloud_extract/cufa.gif)


In [2]:
### SET EXISTING LOCATIONS FOR DATA ###

# Exisiting directory for project. This is were everything thing will placed!
out_directory = "z:\\BoydsGIS\\KYLakes\\"

# Locations of index grids 
las_grid = "z:\\BoydsGIS\\data\\download_grids.gdb\\KY_5k_PointClound_grid"
naip_grid = "z:\\BoydsGIS\\data\\download_grids.gdb\\Kentucky_10K_NAIP"

# NAIP files we are downloadings
naip_prefix = "ky_2ft_naip_2016_"

# Where is the directory with the laszip64.exe
las_tools = "z:\\BoydsGIS\\data\\lidar\\laszip64.exe"

In [3]:
### SET NAMES and VALUES for NEW CONTENT ###

# Create project name - creates a folder in project directory
project = "cumberland"

# Create name of point layer and buffer distance from point
point_name = "cumberland_pt"
buffer_distance = 1000

# Create name of output geodatabase name
out_geodb = "workspace.gdb"

# Create name of downloads folder
downloads = f'{out_directory}{project}\\downloads\\'
lidar = f'{out_directory}{project}\\lidar\\'
lidar_extract = f'{out_directory}{project}\\lidar_extract\\'
lidar_color = f'{out_directory}{project}\\lidar_color\\'

# LAS dataset name and defauly ground class codes
las_dataset = f'{lidar}{project}.lasd'
las_ground = [2]

# Ouput folder name for contours
out_shp = "shapefiles"

# Output parameters for contours
contour_interval = 10
contour_index = 100
contour_name = f"contour_{contour_interval}ft"

In [4]:
# Create folders and copy laszip64.exe
subprocess.run(f'mkdir {out_directory}{project}', shell=True)
subprocess.run(f'mkdir {out_directory}{project}\\downloads', shell=True)
subprocess.run(f'mkdir {out_directory}{project}\\lidar', shell=True)
subprocess.run(f'mkdir {out_directory}{project}\\lidar_color', shell=True)
subprocess.run(f'mkdir {out_directory}{project}\\lidar_extract', shell=True)
subprocess.run(f'mkdir {out_directory}{project}\\{out_shp}', shell=True)
completed = subprocess.run(f'dir {out_directory}{project}', shell=True, stdout=subprocess.PIPE)
print(completed.stdout.decode('UTF-8'))
completed = subprocess.run(f'copy {las_tools[0]}{las_tools[1]} {downloads}', shell=True, stdout=subprocess.PIPE)
print(completed.stdout.decode('UTF-8'))

 Volume in drive Z is Shared Folders
 Volume Serial Number is 0000-0000

 Directory of z:\BoydsGIS\KYLakes\cumberland

04/26/19  09:58 AM    <DIR>          downloads
04/26/19  09:58 AM    <DIR>          lidar
04/26/19  09:58 AM    <DIR>          lidar_color
04/26/19  09:58 AM    <DIR>          lidar_extract
04/26/19  09:58 AM    <DIR>          shapefiles
               0 File(s)              0 bytes
               5 Dir(s)  727,868,899,328 bytes free

z: 
        0 file(s) copied.



In [5]:
# Overwrite if exists
arcpy.env.overwriteOutput = True

# Create project geodatabase
arcpy.CreateFileGDB_management(f'{out_directory}{project}', out_geodb)

# Create project default geodatabase
out_path = f'{out_directory}{project}\\{out_geodb}'
arcpy.env.workspace = out_path

# Create empty point layer
spatial_reference = arcpy.Describe(las_grid).spatialReference
arcpy.CreateFeatureclass_management(out_path, point_name, "POINT", "#", "#", "#", spatial_reference)

<Result 'z:\\BoydsGIS\\KYLakes\\cumberland\\workspace.gdb\\cumberland_pt'>

## Open ArcGIS Pro

What location do you want? Edit the empty point layer by dropping a **single** point somewhere in Kentucky. Save the edit.

In [8]:
# Buffer point
arcpy.Buffer_analysis(point_name, f'{point_name}_{buffer_distance}ft', buffer_distance)

# Create a temp layer to find which LAS files to download
arcpy.Intersect_analysis ([f'{point_name}_{buffer_distance}ft', las_grid], "temp")

# Find URLs and download them and use laszip64.exe to convert 
cursor = arcpy.da.SearchCursor("temp", ['ftppath', 'LASVersion', 'Year'])
i = 0
las_names = []
for row in cursor:
    url = row[0]
    name = url[-12:]
    las_names.append(f'{lidar}{url[-12:-4]}.las')
    print(las_names[i])
    urlretrieve(url, f'{downloads}{name}')
    print(f'{las_tools} -i {downloads}{name} -o {las_names[i]}')
    completed = subprocess.run(f'{las_tools} -i {downloads}{name} -o {las_names[i]}', shell=True, stdout=subprocess.PIPE)
    subprocess.check_call(f'{las_tools} -i {downloads}{name} -o {las_names[i]}', stdout=subprocess.PIPE)
    print(completed.stdout.decode('UTF-8'))
    i += 1
    
# Check for output
print(f'Creating LAS dataset from {las_names}')

# Create LAS dataset
arcpy.CreateLasDataset_management (las_names, las_dataset, "#", "#", spatial_reference, True, True)

arcpy.LasDatasetStatistics_management (las_dataset, "#", f'{out_directory}{project}\\stats.csv', "#", "#", "#")
with open(f'{out_directory}{project}\\stats.csv', encoding='utf-8') as csv:
    reader = pd.read_csv(csv)
    # Create pandas data frame that
    pdData = pd.DataFrame(reader)

pdData[pdData["Category"] == "ClassCodes"]

z:\BoydsGIS\KYLakes\cumberland\lidar\N177E264.las
z:\BoydsGIS\data\lidar\laszip64.exe -i z:\BoydsGIS\KYLakes\cumberland\downloads\N177E264.laz -o z:\BoydsGIS\KYLakes\cumberland\lidar\N177E264.las

Creating LAS dataset from ['z:\\BoydsGIS\\KYLakes\\cumberland\\lidar\\N177E264.las']


,Item,Category,Pt_Cnt,Percent,Z_Min,Z_Max,Intensity_Min,Intensity_Max,Synthetic_Pt_Cnt,Range_Min,Range_Max
10,1_Unclassified,ClassCodes,3877126.0,46.86,543.24,1005.11,0.0,255.0,0.0,NaN,NaN
11,2_Ground,ClassCodes,3740490.0,45.21,543.37,960.07,0.0,255.0,0.0,NaN,NaN
12,7_Low_Point(noise),ClassCodes,4200.0,0.05,543.43,937.48,0.0,255.0,0.0,NaN,NaN
13,9_Water,ClassCodes,647223.0,7.82,542.91,709.70,0.0,255.0,0.0,NaN,NaN
14,10_Reserved,ClassCodes,4833.0,0.06,543.27,716.35,5.0,153.0,0.0,NaN,NaN


# Decide which class codes to add/change

In [9]:
# default ground classes
las_ground = [2, 9]

# Filter for ground points and create DEM and hillshade
arcpy.MakeLasDatasetLayer_management (las_dataset, f'{lidar}ground', las_ground)
arcpy.LasDatasetToRaster_conversion (f'{lidar}ground', f'{project}_dem_5ft', "#", "#", "#", "#", 5)
arcpy.HillShade_3d(f'{project}_dem_5ft', f'{project}_hillshade', 270, 55)

# Create a temp layer to find which NAIP files to download
arcpy.RasterDomain_3d (f'{project}_hillshade', 'domain', 'POLYGON')
arcpy.Intersect_analysis (['domain', naip_grid], "temp")

# Find URLs, download them and extract 
cursor = arcpy.da.SearchCursor("temp", ['ftppath16', 'TileName'])
i = 0
naip_names = []
for row in cursor:
    url = row[0]
    name = row[1]
    naip_names.append(name)
    print(naip_names)
    urlretrieve(url, f'{downloads}{name}.zip')
    with ZipFile(f'{downloads}{name}.zip', 'r') as zip: 
        zip.extractall(f'{downloads}{name}') 
    arcpy.CopyRaster_management (f'{downloads}{name}\\{naip_prefix}{name}.jpg', name)
    i += 1
    
# If multiple NAIPs, then mosaic to new raster and clip
arcpy.Delete_management (f'{out_directory}{project}\\{out_geodb}\\temp')
if len(naip_names) > 1:
    arcpy.MosaicToNewRaster_management (naip_names, out_path, "temp", None, "8_BIT_UNSIGNED", None, 3)
    arcpy.Clip_management ('temp', '#', f'{project}_naip', 'domain')
else:
    arcpy.Clip_management (naip_names[0], '#', f'{project}_naip', 'domain')
arcpy.Delete_management (f'{out_directory}{project}\\{out_geodb}\\temp')    

# Extract LAS points in buffer and colorize
arcpy.ExtractLas_3d (las_dataset, f'{lidar_extract}', f'{point_name}_{buffer_distance}ft', "#", "#", "_extract", "#", "#", False, f'{lidar_extract}temp.lasd')
arcpy.ColorizeLas_3d (f'{lidar_extract}temp.lasd', f'{project}_naip', 'RED Band_1; GREEN Band_2; BLUE Band_3', lidar_color, "_color", "#",  "#",  "#",  "#", True, f'{lidar_color}{project}_rgb.lasd')

# Create DSM of cliffs over 30 feet in 30-ft diameter neighborhood from bare-earth DEM
neighborhood = arcpy.sa.NbrCircle(3,'CELL')
outFocalStat = arcpy.sa.FocalStatistics(f'{project}_dem_5ft', neighborhood, "RANGE")
outFocalStat.save("focal_stats_30ft")
cliffs_over_30ft = arcpy.sa.Con(outFocalStat > 30, outFocalStat)
cliffs_over_30ft.save("cliffs_over_30ft")

# Create contours
neighborhood = arcpy.sa.NbrCircle(5,'CELL')
outFocalStat = arcpy.sa.FocalStatistics(f'{project}_dem_5ft', neighborhood, "MEAN")
outFocalStat.save("avg_elev_30ft")
arcpy.Contour_3d ("avg_elev_30ft", contour_name, contour_interval)

# Create Index contours
code_block = """
def is_index_contour(x):
    if x%contour_index == 0:
        return 1
    else:
        return 0
"""     
arcpy.AddField_management (contour_name, "index", "SHORT")        
arcpy.CalculateField_management(contour_name, "index", "is_index_contour(!CONTOUR!)", "PYTHON3", code_block)

# Clip and zip contours for Mapbox
arcpy.Clip_analysis (contour_name, f'{point_name}_{buffer_distance}ft', f'{out_directory}{project}\\{out_shp}\\{contour_name}_aoi.shp')
shutil.make_archive(f"{out_directory}{project}\\{contour_name}_aoi", 'zip', f'{out_directory}{project}\\{out_shp}\\')

['N089E132']


'z:\\BoydsGIS\\KYLakes\\cumberland\\contour_10ft_aoi.zip'